Un CNN classifieur pour utiliser ELMo. Implémentation de version Tensorflow. Les codes de CNN sont basé sur https://github.com/gaussic/text-classification-cnn-rnn et la partie d'intégration de ELMo bilm est modifié depuis les scripts sur [bilm-tf](https://github.com/allenai/bilm-tf)

In [ ]:
import ast
import re
import os
import keras
from keras.layers import Dense, Embedding, Input
from keras.layers import GRU, Dropout, MaxPooling1D, Conv1D, Flatten, LSTM
from keras.models import Model
from keras import backend as K
import numpy as np
import itertools
from keras.utils import np_utils, Sequence
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from plot_util import plot_confusion_matrix
import time
import h5py
%matplotlib inline
import tensorflow as tf
import os
import sys
import json
import gc
sys.path.append("..")
    
from bilm_tf.bilm import TokenBatcher, BidirectionalLanguageModel, \
weight_layers, dump_token_embeddings


# Model parameters
MAX_FEATURES = 256404
MAX_TEXT_LENGTH = 2000
EMBED_SIZE  = 300
BATCH_SIZE = 64
EPOCHS = 2
VALIDATION_SPLIT = 0.1

# Data Loader

In [ ]:
def filter(s):
    """Output only Chinese, English characters and Arabic numerals"""
    filtrate = re.compile(u'[^\u0030-\u0039\u0041-\u005A\u0061-\u007A\u4E00-\u9FA5]')
    res = filtrate.sub(r' ', s)
    return res

def read_datasets(data_path):
    """read datasets form files"""
    x, y = [], []
    with open(data_path, 'r') as f:
        for line in f:
            try:
                content, label = line.strip().split('__label__')
                if content:
                    x.append(filter(content).replace('  ', ''))
                    y.append(label.replace('\n', ''))
            except:
                pass
                
    return x, y

def read_category(data_path):
    """get classes list and corresponding id list"""
    x, y = read_datasets(data_path)
    categories = np.unique(y)
    cat_to_id = dict(zip(categories, range(len(categories))))
    
    return categories, cat_to_id

def process_datasets(data_path, cat_to_id, max_length=2000, vocab_size):
    """transform words and labels to id, pad the examples"""
    print('Converting data to trainable form...')
    x, y = read_datasets(data_path)
    tokenizer = text.Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(x)
    word_index = tokenizer.word_index
    x_tokenized = tokenizer.texts_to_sequences(x)
    label_id = []
    for i in range(len(x)):
        label_id.append(cat_to_id[y[i]])
        
    x_pad = sequence.pad_sequences(x_tokenized, maxlen=max_length)
    y_pad = keras.utils.to_categorical(label_id, num_classes=len(cat_to_id))
    print('Number of examples counted by x_pad: ' + str(len(x_pad)))
    print('Number of examples counted by y_pad: ' + str(len(y_pad)))
    
    return x_pad, y_pad

def batch_iter(x, y, batch_size=128):
    """generate batch data"""
    data_len = len(x)
    num_batch = int((data_len - 1) / batch_size) 
    if (data_len - 1) % batch_size != 0:
        num_batch = num_batch + 1
    
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x[indices]
    y_shuffle = y[indicies]
    
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]

## CNN Model

In [ ]:
class TCNNConfig(object):
    """CNN parrameters config"""
    embedding_dim = 600  # embedding_size
    seq_length = 2000  # max length for each news
    num_classes = 13  
    num_filters = 32  # number of filters
    kernel_size = 5  # conv kernel size
    vocab_size = 

    hidden_dim = 512  # number of units in fc1
    
    dropout_keep_prob = 0.5 
    learning_rate = 1e-3

    batch_size = 128  
    num_epochs = 2  

    print_per_batch = 100  
    save_per_batch = 10  # save into tensorboard
    

class TextCNN(object):
    """Text Classification, CNN Model"""
    
    def __init__(self, config):
        self.config = config
        # inputs
        self.input_x = tf.placeholder(tf.float32,
                                      [None, self.config.seq_length, self.config.embedding_dim], 
                                      name='input_x')
        self.input_y = tf.placeholder(tf.float32,
                                      [None, self.config.num_classes],
                                      name='input_y')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        self.cnn()
    
    def cnn(self):
        """CNN Model"""
        with tf.name_scope("cnn"):
            # inputs shape : [batch, filter_width, in_channels]
            # filters shape : [filter_width, in_channels, out_channels]
            
            # CNN layer
            conv = tf.layers.conv1d(inputs, 
                                    filters=32,
                                    kernel_size=5,
                                    stride=1, 
                                    padding='same',
                                    name='conv')
            conv_relu = tf.nn.relu(conv)
            conv_dropout = tf.contrib.layers.dropout(conv_relu, keep_prob=self.keep_prob)
            
            # max pooling layer
            mpl = tf.layers.max_pooling1d(conv_dropout,
                                          pool_size=2,
                                          strides=1,
                                          padding='valid',
                                          name='mpl')
        
        with tf.name_scope("score"):
            fc = tf.layers.flatten(mpl)
            fc = tf.layers.dense(fc, units=512, activation=tf.nn.relu, name='fc1')
            fc_dropout = tf.contrib.layers.dropout(fc, keep_prob=self.keep_prob)
            self.logits = tf.layers.dense(fc_dropout, units=13, name='fc2')
            self.y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)

        with tf.name_scope("optimize"):
            # loss function, cross entropy
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,
                                                                    labels=self.input_y)
            self.loss = tf.reduce_mean(cross_entropy)
            # opptimizer
            self.optim = tf.train.AdamOptimizer(learning_rate=self.config.learning_rate).minimize(self.loss)
        
        
        with tf.name_scope("accuracy"):
            # accuracy
            correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)
            self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## Run CNN Model

In [ ]:
import os
import time
from datetime import timedelta
from sklearn import metrics

base_dir = 'datasets/'
train_dir = os.path.join(base_dir, 'train_pure_nolabel.txt')
test_dir = os.path.join(base_dir, 'test_pure_nolabel.txt')
vocab_dir = 'pre_trained_vectors/raw_300/'
save_dir = 'checkpoints/textcnn'

def get_time_diff(start_time):
    """get time passed"""
    end_time = time.time()
    time_diff = end_time - start_time
    return timedelta(seconds=int(round(time_diff)))

def feed_data(x_batch, y_batch, keep_prob):
    feed_dict = {
        model.input_x: x_batch,
        model.input_y: y_batch,
        model.keep_prob: keep_prob
    }
    return feed_dict

def evaluate(sess, x_, y_):
    """evaluate the accuracy and loss on an example"""
    data_len = len(x_)
    batch_eval = batch_iter(x_, y_, 128)
    total_loss = 0.0
    total_acc = 0.0
    for x_batch, y_batch in batch_eval:
        batch_len = len(x_batch)
        feed_dict = feed_data(x_batch, y_batch, 1.0)
        loss, acc = sess.run([model.loss, model.acc], feed_dict=feed_dict)
        total_loss += loss * batch_len
        total_acc += acc * batch_len

    return total_loss / data_len, total_acc / data_len

def train():
    print("Configuring TensorBoard and Saver...")
    # config Tensorboard，delete tensorboard folder for re-training
    tensorboard_dir = 'tensorboard/textcnn'
    if not os.path.exists(tensorboard_dir):
        os.makedirs(tensorboard_dir)
    
    tf.summary.scalar("loss", model.loss)
    tf.summary.scalar("accuracy", model.acc)
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tensorboard_dir)
    
    # config saver
    saver = tf.train.Saver()
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    print("Loading training and validation data...")
    start_time = time.time()
    x, y = process_datasets(train_dir, cat_to_id,
                            max_length=config.seq_length,
                            vocab_size=config.vocab_size)
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1)
    time_diff = get_time_diff(start_time)
    print("Time usage:", time_diff)
    
    # Create session
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        writer.add_graph(session.graph)
        
        print("Training and evaluating...")
        start_time = time.time()
        total_batch = 0 # total batch
        best_acc_val = 0.0 # best validation set accurary
        last_improved = 0 # record last improved batch
        # if more than 1000 batches, we don't get improvement, we terminate training phase
        require_improvement = 1000
        
        flag = False
        for epoch in range(config.num_epochs):
            print("Epoch:", epoch + 1)
            batch_train = batch_iter(x_train, y_train, config.batch_size)
            for x_batch, y_batch in batch_train:
                
                # TODO: add bilm context input
                
                feed_dict = feed_data(x_batch, y_batch, config.dropout_keep_prob)
                
                if total_batch % config.save_per_batch == 0:
                    s = sess.run(merged_summary, feed_dict=feed_dict)
                    writer.add_summary(s, total_batch)
                    
                if total_batch % config.print_per_batch == 0:
                    feed_dict[model.keep_prob] = 1.0
                    loss_train, acc_train = sess.run([model.loss, model.acc],
                                                     feed_dict=feed_dict)
                    loss_val, acc_val = evaluate(sess, x_val, y_val)
                    
                    if acc_val > best_acc_val:
                        # Save the best result
                        best_acc_val = acc_val
                        last_improved = total_batch
                        saver.save(sess=sess, save_path=save_path)
                        improved_str = '*'
                    else:
                        improved_str = ''
                        
                    time_diff = get_time_diff(start_time)
                    msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%},' \
                      + ' Val Loss: {3:>6.2}, Val Acc: {4:>7.2%}, Time: {5} {6}'
                    print(msg.format(total_batch, loss_train, acc_train, loss_val,
                                     acc_val, time_diff, improved_str))
                sess.run(model.optim, feed_dict=feed_dict) # run optimization
                total_batch += 1
                
                if total_batch - last_improved > require_improvement:
                    print("No optimization for a long time, auto-stopping...")
                    flag = True
                    break
            if flag:
                break
                
def test():
    print("Loading test data...")
    start_time = time.time()
    x_test, y_test = process_datasets(test_dir, cat_to_id, conf)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        saver.restore(sess=sess, save_path=save_path)
        
        print("Testing...")
        loss_test, acc_test = evaluate(sess, x_test, y_test)
        msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
        print(msg.format(loss_test, acc_test))
        
        batch_size = config.batch_size
        data_len = len(x_test)
        num_batch = (data_len - 1) / batch_size
        if (data_len - 1) % batch_size != 0:
            num_batch += 1
        
        y_test_cls = np.argmax(y_test, 1)
        y_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)
        for i in range(num_batch):
            start_id = i * batch_size
            end_id = ((i + 1) * batch_size, data_len)
            feed_dict = {
                model.input_x: x_test[start_id:end_id],
                model.keep_prob: 1.0
            }
            y_pred_cls[start_id:end_id] = sess.run(model.y_pred_cls, feed_dict=feed_dict)
        
        print("Precision, Recall and F1-Score...")
        print(metrics.classification_report(y_test_cls, y_pred_cls, target_names=categories))
        
        print("Confusion Matrix...")
        cm = metrics.confusion_matrix(y_test_cls, y_pred_cls)
        print(cm)
        
        time_diff = get_time_diff(start_time)
        print("Time usage: ", time_diff)

        
def run_cnn(argv):
    if argv not in ['train', 'test']:
        raise ValueError("Wrong argument.")
        
    print("Configuring CNN Model...")
    config = TCNNConfig()
    